<img src="https://sedu.es.gov.br/Media/sedu/_Profiles/c4d8c6e6/982b8e1d/inscricoes-enem-2021-1.png?v=637783383873645750" alt="jupyter" style="width: 300px;" align="right"/>

# Projeto Final Big Data Science

Vagner Sanches Vasconcelos    
vsvasconcelos@gmail.com

#  <a name="resumo"> RESUMO </a>

Este projeto tem como objetivo estudar os dados do Exame Nacional do Ensino Médio (Enem) de 2021.

#  <a name="indice">  Índice </a>

* [Resumo](#resumo)
* [Índice](#indice)
* [1. Definição do Problema](#secao_1)
* [2. Obtendo os Dados](#secao_2)
* [3. Limpando os Dados](#secao_3)
* [4. Análise Exploratória dos Dados](#secao_4)
* [5. Formulação de Hipoteses e Testes ](#secao_5)
* [6. Construindo e Testando Modelos](#secao_6) 
* [Conclusão](#conclusao)

#  <a name="secao_1"> 1. Definição do Problema </a>
[voltar ao índice](#indice)

O Enem foi instituído em 1998, com o objetivo de avaliar o desempenho escolar dos estudantes ao término da educação básica. Em 2009, o exame aperfeiçou sua metodologia e passou a ser utilizado como mecanismo de acesso à educação superior.   
Neste trabalho analisaremos os últimos dados disponiveis que são do ano de 2021.    
O objetivo é descobrir a participação das pessoas com 25 anos ou mais neste processo.

#  <a name="secao_2"> 2. Obtendo os dados </a>
[voltar ao índice](#indice)

Os [microdados](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados
) do Inep reúnem um conjunto de informações detalhadas relacionadas às pesquisas, aos exames e avaliações do Instituto.    
No [portal do Instituto](https://download.inep.gov.br/microdados/microdados_enem_2021.zip)  é possível baixar os dados disponíveis, sendo o ano de 2021 os mais atuais.    
Depois de baixado e descompactado, tem-se uma série de pastas, sendo que na pasta "DADOS", temos o arquivo "MICRODADOS_ENEM_2021.csv, que é o conjunto de dados, e possui tamanho de 1,5GB.   
Devido o tamanho do arquivo, foi utilizado a biblioteca de computação paralela [Dask](https://www.dask.org/) para sua leitura.

In [1]:
#Importando as bibliotecas
import pandas as pd
import dask.dataframe as dd 
from matplotlib import pyplot as plt
from dask.diagnostics import ProgressBar
import numpy as np

In [2]:
#Carregando a base de dados num dataframa Dask
df = dd.read_csv("./microdados_enem_2021/DADOS/MICRODADOS_ENEM_2021.csv",sep = ';', encoding = 'latin1',
                 dtype={'Q005': 'float64'})

In [3]:
#Visão geral do dataframe
df.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B


__Observamos as 5 primeiras linhas do dataframe, que possui 76 colunas (variáveis), sendo a 1ª "NU_INSCRICAO" e a última "Q025".__

In [7]:
#Verificando todas as variáveis
df.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

__Na pasta "Dicionario", temos o arquivo "Dicionário_Microdados_Enem_2021.xlsx" que detalha a descrição de cada variável, bem como seus valores, tamanho e tipo.__

Exemplos de algumas variáveis e suas descrições:   
Q001 - Até que série seu pai, ou o homem responsável por você, estudou?    
Q002 - Até que série sua mãe, ou a mulher responsável por você, estudou?   
Q025 - Na sua residência tem acesso à Internet?

In [4]:
#Quantidade de registros
df.shape[0].compute()

3389832

__A base de dados possui mais de 3,3 milhões de registros.__

#  <a name="secao_3"> 3. Limpando os dados </a>
[voltar ao índice](#indice)

#  <a name="secao_4"> 4. Análise Exploratória dos Dados </a>
[voltar ao índice](#indice)

### Quantas pessoas com 25 anos ou mais fizeram a prova?

In [20]:
maiores_25_anos = (df['TP_FAIXA_ETARIA'] >= 11) & (df['TP_FAIXA_ETARIA'] <= 20)
maiores_25_anos.value_counts().compute()

False    2806505
True      583327
Name: TP_FAIXA_ETARIA, dtype: int64

__Mais de meio milhão de pessoas__

### Qual o sexo destas pessoas?

In [22]:
df[maiores_25_anos]['TP_SEXO'].value_counts().compute()

F    346454
M    236873
Name: TP_SEXO, dtype: int64

__Cerca de 346 mil são mulheres e 237 mil são homens__

### Quais as raças destas pessoas?

In [24]:
df['TP_COR_RACA'] = df['TP_COR_RACA'].map({0: 'Não declarado',
                                           1: 'Branca',
                                           2: 'Preta',
                                           3: 'Parda',
                                           4: 'Amarela',
                                           5: 'Indígena',
                                           6: 'Não dispões da informação'})

In [25]:
df[maiores_25_anos]['TP_COR_RACA'].value_counts().compute()

Parda                        282556
Branca                       177405
Preta                         95645
Não declarado                 12865
Amarela                       10790
Indígena                       4061
Não dispões da informação         5
Name: TP_COR_RACA, dtype: int64

#  <a name="secao_5"> 5. Formulação de Hipoteses e Testes </a>
[voltar ao índice](#indice)

#  <a name="secao_6"> 6. Construindo e Testando Modelos </a>
[voltar ao índice](#indice)

#  <a name="conclusao"> Conclusões </a>
[voltar ao índice](#indice)